In [1]:
import os
import util
import model_builder
import numpy as np
import pandas as pd
import tensorflow as tf

Working plan - Since it's seen that MobileNetV2 + GRU gave an adequately good result, I want to try the architecture (the best alone or the top 5) on some variations:
1. on the same videos decomposed into more number of frames (I did 16, try 32 and 64)
2. on a different dataset (I used DFD, try CelebDF)
3. using efficientnet or some other pretrained model (so adjust image sizes accordingly)

Make generic functions so that any data, any number of frames, and any pretrained model can be used. Save all the best ones.

In [2]:
base_dir = r'data'
data_sources = ['DFD', 'CelebDF']
num_frames = [16, 32, 64, 128, 256]

In [3]:
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)

# DFD More Frames

In [4]:
labels1, classifier1, (classifier_loss1, classifier_acc1) = model_builder.train_test_classifier(
    data_dir=os.path.join(base_dir, 'DFD'),
    num_frames=64)

Image Model: MobileNetV2, Image Size: (224, 224)
TRAIN set: 140 videos
VAL set: 30 videos
TEST set: 30 videos
TRAIN set: 140 videos
VAL set: 30 videos
TEST set: 30 videos
Video data extraction and splitting completed.

Shapes:
X_train: (280, 64, 224, 224, 3)
X_val: (60, 64, 224, 224, 3)
X_test: (60, 64, 224, 224, 3)
y_train: (280,)
y_val: (60,)
X_test: (60,)
560/560 ━━━━━━━━━━━━━━━━━━━━ 137s 240ms/step
Embeddings shape: (17920, 1280)
120/120 ━━━━━━━━━━━━━━━━━━━━ 31s 243ms/step
Embeddings shape: (3840, 1280)
120/120 ━━━━━━━━━━━━━━━━━━━━ 31s 243ms/step
Embeddings shape: (3840, 1280)
Temporal model defined.


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 256)                 │       1,575,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,576,193 (6.01 MB)

 Trainable params: 1,576,193 (6.01 MB)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 6s 242ms/step - accuracy: 0.5071 - loss: 0.7692 - val_accuracy: 0.4000 - val_loss: 0.7579
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - accuracy: 0.4643 - loss: 0.7439 - val_accuracy: 0.5000 - val_loss: 0.7462
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - accuracy: 0.4893 - loss: 0.7417 - val_accuracy: 0.5167 - val_loss: 0.7417
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 167ms/step - accuracy: 0.5036 - loss: 0.7068 - val_accuracy: 0.5000 - val_loss: 0.7386
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 168ms/step - accuracy: 0.4750 - loss: 0.7421 - val_accuracy: 0.4833 - val_loss: 0.7359
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 169ms/step - accuracy: 0.5107 - loss: 0.7100 - val_accuracy: 0.4667 - val_loss: 0.7331
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 168ms/step - accuracy: 0.4964 - loss: 0.7068 - val_accuracy: 0.4833 - val_loss: 0.7303
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 171ms/step - accuracy: 0.5286 - loss: 0.7041 - val_accuracy: 0.466

In [5]:
labels1

array(['real', 'fake'], dtype='<U4')

In [6]:
print(f"DFD with more Frames-\nLoss: {classifier_loss1}\nAccuracy: {classifier_acc1}")

DFD with more Frames-
Loss: 0.746757447719574
Accuracy: 0.5166666507720947


* The same architecture (2 GRUs) that gave 66.7% accuracy for DFD with 16 frames is only giving **51.7%** when applied on DFD videos split into 64 frames. This is probably because there's more temporal dependencies to capture here and 2 GRUs aren't sufficient for that.

# DFD More Frames GRUs & Dense Layers

In [10]:
data_dir = os.path.join(base_dir, 'DFD')
os.path.dirname(data_dir), os.path.basename(data_dir).lower()

('data', 'dfd')

In [4]:
labels2, classifier2, (classifier_loss2, classifier_acc2) = model_builder.train_test_classifier(
    data_dir=os.path.join(base_dir, 'DFD'),
    num_frames=64,
    num_gru=2, num_dense=1, batchnorm=True)

Image Model: MobileNetV2, Image Size: (224, 224)
Video data extraction and splitting completed.

Shapes:
X_train: (280, 64, 224, 224, 3)
X_val: (60, 64, 224, 224, 3)
X_test: (60, 64, 224, 224, 3)
y_train: (280,)
y_val: (60,)
X_test: (60,)
560/560 ━━━━━━━━━━━━━━━━━━━━ 136s 237ms/step
Embeddings shape: (17920, 1280)
120/120 ━━━━━━━━━━━━━━━━━━━━ 29s 236ms/step
Embeddings shape: (3840, 1280)
120/120 ━━━━━━━━━━━━━━━━━━━━ 30s 246ms/step
Embeddings shape: (3840, 1280)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 64, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 64, 256)             │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,609,856 (6.14 MB)

 Trainable params: 1,609,344 (6.14 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 128)                 │       1,609,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,609,985 (6.14 MB)

 Trainable params: 1,609,473 (6.14 MB)

 Non-trainable params: 512 (2.00 KB)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 182ms/step - accuracy: 0.4786 - loss: 0.8555 - val_accuracy: 0.5167 - val_loss: 0.7156
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step - accuracy: 0.5036 - loss: 0.8782 - val_accuracy: 0.5167 - val_loss: 0.7125
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - accuracy: 0.5286 - loss: 0.8430 - val_accuracy: 0.5000 - val_loss: 0.7095
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step - accuracy: 0.5000 - loss: 0.8823 - val_accuracy: 0.5000 - val_loss: 0.7070
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 150ms/step - accuracy: 0.5000 - loss: 0.8448 - val_accuracy: 0.5167 - val_loss: 0.7049
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 147ms/step - accuracy: 0.5143 - loss: 0.8433 - val_accuracy: 0.5167 - val_loss: 0.7032
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 144ms/step - accuracy: 0.5500 - loss: 0.8031 - val_accuracy: 0.5333 - val_loss: 0.7015
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 142ms/step - accuracy: 0.5786 - loss: 0.7863 - val_accuracy: 0.5333 - v

In [5]:
labels2

array([0, 1], dtype=int64)

In [6]:
print(f"DFD with more Frames and 4 GRUs-\nLoss: {classifier_loss2}\nAccuracy: {classifier_acc2}")

DFD with more Frames and 4 GRUs-
Loss: 0.7296609878540039
Accuracy: 0.5


* With both 4 GRUs and 2 GRUs + 1 Dense + BatchNormalization, only **50%** accuracy can be obtained. Perhaps the added temporal dependencies from more number of frames adds more load on the already small dataset. So, it appears 16 frames was the best case.

# DFD Xception

In [5]:
labels3, classifier3, (classifier_loss3, classifier_acc3) = model_builder.train_test_classifier(
    data_dir=os.path.join(base_dir, 'DFD'),
    num_frames=16,
    img_model_name='Xception')

Image Model: Xception, Image Size: (299, 299)
TRAIN set: 140 videos
VAL set: 30 videos
TEST set: 30 videos
TRAIN set: 140 videos
VAL set: 30 videos
TEST set: 30 videos
Video data extraction and splitting completed.

Shapes:
X_train: (280, 16, 299, 299, 3)
X_val: (60, 16, 299, 299, 3)
X_test: (60, 16, 299, 299, 3)
y_train: (280,)
y_val: (60,)
X_test: (60,)
140/140 ━━━━━━━━━━━━━━━━━━━━ 244s 2s/step
Embeddings shape: (4480, 2048)
30/30 ━━━━━━━━━━━━━━━━━━━━ 53s 2s/step 
Embeddings shape: (960, 2048)
30/30 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step 
Embeddings shape: (960, 2048)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 16, 256)             │       1,771,008 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,165,760 (8.26 MB)

 Trainable params: 2,165,760 (8.26 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 256)                 │       2,165,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,166,017 (8.26 MB)

 Trainable params: 2,166,017 (8.26 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 112ms/step - accuracy: 0.4857 - loss: 0.7206 - val_accuracy: 0.5000 - val_loss: 0.6888
Epoch 2/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.5500 - loss: 0.7069 - val_accuracy: 0.5000 - val_loss: 0.6871
Epoch 3/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.5393 - loss: 0.6992 - val_accuracy: 0.5000 - val_loss: 0.6855
Epoch 4/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.5357 - loss: 0.6942 - val_accuracy: 0.5000 - val_loss: 0.6843
Epoch 5/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.5321 - loss: 0.6998 - val_accuracy: 0.5167 - val_loss: 0.6832
Epoch 6/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.5679 - loss: 0.6794 - val_accuracy: 0.4833 - val_loss: 0.6823
Epoch 7/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.5393 - loss: 0.7015 - val_accuracy: 0.5000 - val_loss: 0.6816
Epoch 8/500
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.5929 - loss: 0.6719 - val_accuracy: 0.5167 - val_loss

In [6]:
labels3

array(['real', 'fake'], dtype='<U4')

In [7]:
print(f"DFD using Xception-\nLoss: {classifier_loss3}\nAccuracy: {classifier_acc3}")

DFD using Xception-
Loss: 0.6473307013511658
Accuracy: 0.6166666746139526


* The exact same temporal model that gave 66.7% accuracy with MobileNetV2 was only able to achieve **61.7%** with Xception. Hence, it appears that MobileNetV2 could capture more spatial dependencies.

# CelebDF (best pretrained model) & (best num frames)

In [4]:
labels4, classifier4, (classifier_loss4, classifier_acc4) = model_builder.train_test_classifier(
    data_dir=os.path.join(base_dir, 'CelebDF'),
    num_frames=16,
    img_model_name='Xception')

Image Model: Xception, Image Size: (299, 299)
Video data extraction and splitting completed.

Shapes:
X_train: (812, 16, 299, 299, 3)
X_val: (174, 16, 299, 299, 3)
X_test: (174, 16, 299, 299, 3)
y_train: (812,)
y_val: (174,)
X_test: (174,)
406/406 ━━━━━━━━━━━━━━━━━━━━ 688s 2s/step 
Embeddings shape: (12992, 2048)
87/87 ━━━━━━━━━━━━━━━━━━━━ 141s 2s/step
Embeddings shape: (2784, 2048)
87/87 ━━━━━━━━━━━━━━━━━━━━ 142s 2s/step
Embeddings shape: (2784, 2048)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 16, 256)             │       1,771,008 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,165,760 (8.26 MB)

 Trainable params: 2,165,760 (8.26 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential (Sequential)              │ (None, 256)                 │       2,165,760 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,166,017 (8.26 MB)

 Trainable params: 2,166,017 (8.26 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - accuracy: 0.4926 - loss: 0.7103 - val_accuracy: 0.5345 - val_loss: 0.6915
Epoch 2/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.5382 - loss: 0.6959 - val_accuracy: 0.5230 - val_loss: 0.6917
Epoch 3/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.5012 - loss: 0.7012 - val_accuracy: 0.5402 - val_loss: 0.6911
Epoch 4/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.5074 - loss: 0.6982 - val_accuracy: 0.5460 - val_loss: 0.6908
Epoch 5/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.5431 - loss: 0.6957 - val_accuracy: 0.5230 - val_loss: 0.6910
Epoch 6/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.5345 - loss: 0.6898 - val_accuracy: 0.5230 - val_loss: 0.6906
Epoch 7/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step - accuracy: 0.5234 - loss: 0.6985 - val_accuracy: 0.5345 - val_loss: 0.6907
Epoch 8/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.5517 - loss: 0.6866 - val_accuracy: 0.

In [5]:
labels4

array([0, 1], dtype=int64)

In [7]:
print(f"CelebDF with 16 Frames, Xception, 2GRUs -\nLoss: {classifier_loss4}\nAccuracy: {classifier_acc4}")

CelebDF with 16 Frames, Xception, 2GRUs -
Loss: 0.692363440990448
Accuracy: 0.5287356376647949


* The same architecture that gave 61.7% accuracy for DFD only gave **52.9%** accuracy for CelebDF. The size of the dataset is slightly bigger than DFD but perhaps 16 frames & 2 GRUs aren't sufficient to capture the temporal dependencies or XceptionNet isn't good at capturing the spatial dependencies of this dataset.

In [9]:
labels5, classifier5, (classifier_loss5, classifier_acc5) = model_builder.train_test_classifier(
    data_dir=os.path.join(base_dir, 'CelebDF'),
    num_frames=16,
    img_model_name='MobileNetV2')

Image Model: MobileNetV2, Image Size: (224, 224)
TRAIN set: 406 videos
VAL set: 87 videos
TEST set: 87 videos
TRAIN set: 406 videos
VAL set: 87 videos
TEST set: 87 videos
Video data extraction and splitting completed.

Shapes:
X_train: (812, 16, 224, 224, 3)
X_val: (174, 16, 224, 224, 3)
X_test: (174, 16, 224, 224, 3)
y_train: (812,)
y_val: (174,)
X_test: (174,)
406/406 ━━━━━━━━━━━━━━━━━━━━ 103s 246ms/step
Embeddings shape: (12992, 1280)
87/87 ━━━━━━━━━━━━━━━━━━━━ 22s 236ms/step
Embeddings shape: (2784, 1280)
87/87 ━━━━━━━━━━━━━━━━━━━━ 23s 244ms/step
Embeddings shape: (2784, 1280)


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_2 (GRU)                          │ (None, 16, 256)             │       1,181,184 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 256)                 │         394,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,575,936 (6.01 MB)

 Trainable params: 1,575,936 (6.01 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_2 (Sequential)            │ (None, 256)                 │       1,575,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,576,193 (6.01 MB)

 Trainable params: 1,576,193 (6.01 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.4914 - loss: 0.7423 - val_accuracy: 0.5230 - val_loss: 0.7068
Epoch 2/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.5246 - loss: 0.7183 - val_accuracy: 0.5115 - val_loss: 0.7057
Epoch 3/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.5012 - loss: 0.7225 - val_accuracy: 0.5172 - val_loss: 0.7048
Epoch 4/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.5259 - loss: 0.7126 - val_accuracy: 0.5172 - val_loss: 0.7043
Epoch 5/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.5012 - loss: 0.7258 - val_accuracy: 0.5287 - val_loss: 0.7033
Epoch 6/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.5049 - loss: 0.7180 - val_accuracy: 0.5345 - val_loss: 0.7026
Epoch 7/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.5259 - loss: 0.7070 - val_accuracy: 0.5345 - val_loss: 0.7025
Epoch 8/500
26/26 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step - accuracy: 0.5333 - loss: 0.7077 - val_accuracy: 0.

In [11]:
labels5

array(['real', 'fake'], dtype='<U4')

In [10]:
print(f"CelebDF with 16 Frames, MobileNetV2, 2GRUs -\nLoss: {classifier_loss5}\nAccuracy: {classifier_acc5}")

CelebDF with 16 Frames, MobileNetV2, 2GRUs -
Loss: 0.7166829705238342
Accuracy: 0.4482758641242981


* The best performing model and setup for DFD dataset gave just **44.8%** accuracy for the CelebDF dataset.